In [1]:
import json
import math
import re
import time

from lxml import html
from bs4 import BeautifulSoup
import requests

In [2]:
href = "https://www.1111.com.tw/job-bank/job-index.asp?si=1&ks=程式設計師&d0=140200&ps=100&page=1"

In [3]:
res = requests.get(href)
soup = BeautifulSoup(res.text,"lxml")
numberReg = re.compile(r'(\d+) 頁')
mo = numberReg.search(soup.select_one("div.pagedata").text)
pageNum = int(mo.group(1))
pageNum

16

In [4]:
linkList = []
jobList = soup.select("div.jbInfo > div > h3 > a")
for job in jobList:
    link = "http:"+job["href"]
    linkList.append(link)
link = linkList[2]

In [5]:
jobInfo = dict()
res2 = requests.get(link)
soup2 = BeautifulSoup(res2.text, 'lxml')
tree = html.fromstring(res2.content)

In [6]:
x = soup2.select("ul.dataList")[1].select("div.listTitle")[-1]
x.find_next_sibling().text

'1.3年以上期貨或證券系統開發或維護經驗\xa02.熟COBOL者佳\xa03.細心負責、有良好的溝通技巧，可獨立作業'

In [7]:
jobInfo["title"] = soup2.select_one("h1").contents[0]
jobInfo["company"] = soup2.select_one("li.ellipsis").text
jobInfo["content"] = soup2.select_one("li.paddingLB").text.replace("\xa0","")
dateReg = re.compile(r'：(\d+\/\d+\/\d+)')
mo2 = dateReg.search(soup2.select_one("span.update").text)
jobInfo["post_date"] = mo2.group(1)
skills = tree.xpath("//div[contains(text(), '電腦專長：')]/following-sibling::div")
if skills != [] :
    jobInfo["skills"] = skills[0].text
else:
    jobInfo["skills"] = ""
other_condition_tag = tree.xpath("//div[contains(text(), '附加條件：')]/following-sibling::div")
if other_condition_tag != []:
    other_condition= "".join([one.text for one in other_condition_tag[0]] )
    jobInfo["other_condition"] = other_condition.replace("\xa0","").replace("\u3000","")
else:
    jobInfo["other_condition"] = ""

In [8]:
jobInfo

{'company': '華南永昌綜合證券股份有限公司',
 'content': '1.TANDEM COBOL程式設計2.負責軟體之分析、設計以及程式撰寫',
 'other_condition': '1.3年以上期貨或證券系統開發或維護經驗2.熟COBOL者佳3.細心負責、有良好的溝通技巧，可獨立作業',
 'post_date': '2017/7/17',
 'skills': '',
 'title': '程式設計師'}

In [13]:
now=time.ctime()
# time.sleep(5)
# end=time.localtime()
now 
# end - now

'Tue Jul 18 09:00:00 2017'